# Fouille de texte avec Python

---

## Introduction

Dans cette étape, nous allons analyser les corpus textuels. La chaîne de traitement est la suivante:
- les quatre fichiers sont ouverts et chargés
- le texte brut est structuré dans des dictionnaires (`dict`) Python: par détection de motifs, les différentes entrées sont séparées; les parties de chaque entrée sont elles aussi extraites
- à partir de ces dictionnaires, les données utiles pour construire nos graphiques sont extraites et on calcule des statistiques
- des graphiques sont créées pour analyser le corpus avec la librairie Python `Plotly`

Pour rappel, la question qu'on se pose est:

> Quelle est la représentation de cinq genres littéraires du XVIIIe siècle (poésie, théâtre, roman, littérature d'idées) dans un corpus datant du XIXe siècle de catalogues de vente de manuscrits ?

---

## Poser les bases

On commence par faire quelques petites choses utiles pour la suite du notebook.

### Importer les librairies utilisées

En Python, certaines fonctionnalités sont disponibles de base, mais la plupart sont organisées dans des *libraires*. Une librairie est un ensemble de fonctions rassemblées ensemble avec un but spécifique: visualisations, calcul... Certaines librairies sont disponibles de base en Python, d'autres sont développées par des tiers et doivent être installées. Une librairie peut aussi être appelée `module` ou `package`.

In [4]:
# syntaxe de base: `import <nom de librairie>`
import os  # la librairie pour gérer les chemins de fichiers
import re  # librairie pour les expressions régulières (détection de motifs dans le texte)import plotly.graph_objects as go  # librairie pour générer des graphiques

# importer une partie d'une librairie
from statistics import median  # on importe la fonction `median` dans la librairie `statistics` pour calculer une valeur médiane
# la librairie plotly est très lourde => on importe seulement une partie: `graph_objects`. 
# on donne à `graph_objects` l'alias `go` avec `as`. Dans notre code, on appellera `go` pour
# utiliser le module `plotly.graph_objects`.
import plotly.graph_objects as go

### Définir des chemins de fichiers

Le chemin vers un fichier peut être écrit comme une chaîne de caractère:

```python
# `./` indique un chemin relatif: on part du fichier actuel
filepath = "./chemin/vers/le/fichier.txt"  
```

Le problème, c'est que différents systèmes utilisent différentes manières d'écrire les chemins. Un chemin peut aussi changer. C'est mieux d'utiliser une librairie pour construire les chemins, comme le module `os.path`. On définit des chemins vers les dossiers utilisés. 

La syntaxe est un peu douloureuse. Pour décomposer la ligne ci-dessous, on part de l'élément le plus à l'intérieur pour aller vers celui le plus à l'extérieur:
- `os.path.dirname()`: le nom d'un dossier. Par défaut, c'est le chemin du dossier actuel qui est utilisé.
- `os.path.abspath()`: le chemin absolu vers le dossier dans lequel se trouve `__file__`: le chemin absolu du fichier actuel.

In [9]:
current_directory = os.path.abspath(os.path.dirname(""))  # le chemin absolu du dossier où se trouve le fichier actuel

Ensuite, on crée un chemin vers les dossiers d'entrée et de sortie Et on crée le dossier de sortie. On utilise:
- `os.path.join()` qui permet de créer un chemin entre les dossiers/fichiers en arguments de la fonction. Ici, on crée deux chemins:
    - entre le dossier actuel et le dossier `in`, soit le dossier qui contient les fichiers texte utilisés
    - entre le dossier actuel et le dossier `out`, soit le dossier qui contientra nos résultats
- `os.path.isdir()` pour vérifier si un dossier existe
- `os.makedirs()` pour créer les dossiers nécessaires

In [11]:
# chemin vers les dossiers d'entrée et de sortie
in_directory = os.path.join(current_directory, "in")
out_directory = os.path.join(current_directory, "out")
print(in_directory)

# créer le dossier de sortie
if not os.path.isdir(out_directory):
    os.makedirs(out)

/home/paulhector/documents/cours/ens2023_fouille_de_texte/in


---

## Ouvrir les fichiers dans `in/` et lire leur contenu

Pour lire le contenu d'un fichier en Python, 2 étapes sont nécessaires: il faut d'abord ouvrir un fichier, ensuite lire son contenu.

- pour ouvrir, on utilise `open(fichier_à_ouvrir)`
- la syntaxe `with open(...) as fh` permet d'ouvrir un fichier et de l'assigner à une variable
- `fh.read()` permet d'accéder au contenu du fichier.

On ouvre les fichiers et on lit la première entrée de chaque fichier. Pour faire plus simple, on définit une fonction pour lire tous nos fichiers. Cela évite de réécrire 4 fois le même bout de code.

In [30]:
def read_input_file(file_name):
    """
    `def` indique qu'on définit une fonction. 
    la syntaxe est: def nom_de_la_fonction(liste, de, nos, arguments).
    """
    with open( os.path.join(in_directory, file_name), mode="r" ) as fh:  # on ouvre le fichier. le `mode= "r"` indique qu'on ouvre le fichier en lecture.
        corpus = fh.read()  # on lit le fichier
    # quand on "retourne" une variable avec `return`, on permet d'utiliser 
    # sa valeur en dehors de la fonction. les variables retournées sont le 
    # résultat de la fonction, toutes les autres sont supprimées quand la fonction se termine
    return corpus


corpus_idees = read_input_file("catalogue_idees.txt")
corpus_roman = read_input_file("catalogue_roman.txt")
corpus_poeme = read_input_file("catalogue_poeme.txt")
corpus_theatre = read_input_file("catalogue_theatre.txt")

# on imprime un de nos corpus. que remarquez-vous sur sa structure? 
print(corpus_idees)

Voltaire
Écrit en 1763-09-15. Vendu en 1879.
Dimensions: 1.5 pages.
 L. s., écrite par Wagnière; Ferney, 15 sept. 1763, 1 p. 1/2 in-4. 


Voltaire
Écrit en 1756-02-29. Vendu en 1879.
Dimensions: 1.0 pages.
 L. aut., sig. V., à M. de Gaufecour; Monrion, 29 fév. 1756, 1 p. pl. in-4. Belle et intéressante lettre.


Voltaire
Écrit en 1758-09-17. Vendu en 1879.
Dimensions: 4.0 pages.
 L. aut., sig. V. (à Tiriot); les Délices, 17 sept. 1758, 4 p. pl. pet. in-4. 


Voltaire
Écrit en 1760-11-12. Vendu en 1879.
Dimensions: 0.75 pages.
 L. aut., sig. V., au comte d'Argental; 12 nov. 1760, 3/4 de p. in-4, cachet. Recommandation en faveur d'un comédien.


Voltaire
Vendu en 1879.
Dimensions: 23.0 pages.
Fragment de la tragédie de Zaire, avec des corrections autographes, 23 p. in-fol. 


VOLTAIRE
Écrit en 1751-05-08. Vendu en 1897.
Dimensions: 2.5 pages.
 L. a. s. V. à M. de Vaux ; Potsdam, 8 mai 1751, 2 p. 1/2 in-4. 


Voltaire
Écrit en 1719. Vendu en 1879.
Dimensions: 4.0 pages.
 Pièce de vers aut

---

## Structurer les corpus

Dans leur forme actuelle, il est difficile de faire sens de nos 4 corpus: ils ne sont pas structurés, et un ordinateur lit de manière bête et méchante: il lit lettre à lettre, ne fait pas la différence entre les mots, les lignes et encore moins les différentes entrées de catalogue!

Pour pouvoir exploiter notre corpus, on va donc devoir le structurer. N'importe quel travail de structuration, de lecture du texte commence par une vraie lecture, humaine du texte: c'est comme ça que l'on reconnaît sa structure. On peut dire que la structure d'une entrée est la suivante:

```
Nom de l'auteur.ice
Écrit en [date]. Vendu en [date].
Dimensions: [nombre décimal].
[description en texte libre]
Prix: [nombre décimal] [monnaie] (en francs constants 1900: [nombre décimal].  # ligne optionnelle
[saut de ligne]
[saut de ligne]
```

Une fois qu'on a reconnu cette structure, la première étape est de la rendre explicite pour l'ordinateur: du texte brut, on doit passer à un format structuré où l'ordinateur comprenne la différence entre les entrées et entre les différentes parties d'une entrée.

On va chercher à représenter chaque entrée par **une liste (`list`) de dictionnaires (`dict`)**:
- **`list`**: une liste est une série de valeurs séparées par des virgules (`,`) qui s'écrit entre `[]`. Elle peut contenir n'importe quel type de données.
    - exemple: `[ 1, "deux",, ["liste", "imbriquée"]]`
- **`dict`**: un dictionnaire permet d'associer deux éléments. 
    - Comme dans un vrai dictionnaire (fait de couples mot-définition), un premier élément (la clé) est associée à une deuxième (la valeur). La valeur peut être n'importe quel type de données.
    - Il s'écrit ainsi: `{"clé": "valeur"}`.
    - exemple: `{"rouge": ["fraise", "tomate"], "blanc": "neige";}`